# Longitudinal Plane and Instability Simulation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xobjects as xo
import xpart as xp
import xtrack as xt
import xwakes as xw

context = xo.ContextCpu(omp_num_threads="auto")
%config InlineBackend.figure_format = "retina"

### Load Machine and Twiss

In [ ]:
# Import in Xsuite
env = xt.Environment.from_json("pimm.json")
env.vars.load_json("pimm_strengths.json")

line = env.ring
line.configure_bend_model(num_multipole_kicks=5)
twiss4d = line.twiss(method="4d")

In [ ]:
# Insert horizontal aperture limitation to see losses
line.discard_tracker()
line.append_element(name="aperture", element=xt.LimitRect(min_x=-0.05, max_x=0.05));

### Switch on the RF

In [ ]:
# Power the RF Cavity's variables
env.vars["vrf"] = 10e3  # 10kV
env.vars["frf"] = 1 / twiss4d.T_rev0  # h=1

In [ ]:
# Now perform a 6D Twiss
twiss0 = line.twiss()
twiss0.qs

## Inspect longitudinal phase space

In [ ]:
# Track a set of particles with different momenta
p = line.build_particles(delta=np.linspace(-1e-2, 1e-2, 101))
line.track(p, num_turns=1500, turn_by_turn_monitor=True, with_progress=True)
mon = line.record_last_track

In [ ]:
plt.figure()
plt.plot(mon.zeta.T, mon.delta.T * 1e3, c="C0", lw=1)
plt.xlim(-40, 40)
plt.xlabel(r"$\zeta$ [m]")
plt.ylabel(r"$\delta$ [10$^{-3}$]")
plt.title("Longitudinal Phase Space")
plt.show()

## Install Transverse Wakefields

In [ ]:
# Install a resonator wakefield to represent the impedance in the line
wakefield = xw.WakeResonator(kind="dipolar_x", r=100e6, f_r=1.3e6, q=1.0)  # Shunt impedance
wakefield.configure_for_tracking(zeta_range=(-20, 20), num_slices=20)
line.append("wf", wakefield)

### Generate a Bunch Matched to the Bucket and Optics

In [ ]:
%%capture
line.build_tracker(_context=context)

bunch = xp.generate_matched_gaussian_bunch(
    line=line,
    num_particles=10_000,
    total_intensity_particles=1e11,
    nemitt_x=2e-6,
    nemitt_y=2e-6,
    sigma_z=10.0,
    _context=context
)
bunch.circumference = twiss0.circumference  # required by xwakes

In [ ]:
# Apply an initial kick of 1 mm
bunch.x += 1e-3
bunch0 = bunch.copy(_context=context)  # keep initial state

### Simulate 1000 Turns

In [ ]:
# Define quantities to be logged during tracking
def compute_x_mean(line: xt.Line, particles: xt.Particles) -> float:
    nplike = particles._context.nplike_lib
    bunch.hide_lost_particles()
    x_average = nplike.mean(particles.x)
    bunch.unhide_lost_particles()
    return x_average


def compute_intensity(line: xt.Line, particles: xt.Particles) -> float:
    nplike = particles._context.nplike_lib
    bunch.hide_lost_particles()
    intensity = nplike.sum(particles.weight)
    bunch.unhide_lost_particles()
    return intensity


# Create a logger to compute these at each turn
track_log = xt.Log(x_mean=compute_x_mean, intensity=compute_intensity)

In [ ]:
# Track!
line.enable_time_dependent_vars = True
line.track(bunch, log=track_log, num_turns=1000, with_progress=5)

In [ ]:
# Plot logged data
x_mean = np.array(line.log_last_track["x_mean"])
intensity = np.array(line.log_last_track["intensity"])

plt.figure(figsize=(7, 5))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(x_mean, label=rf"$Q_x^' = {twiss0.dqx:.2f}, Q_y^' = {twiss0.dqy:.2f}$")
ax1.set_ylabel(r"$x_{\mathrm{centroid}}$ [m]")

ax2 = plt.subplot(2, 1, 2, sharex=ax1)
ax2.plot(intensity * 1e-11, label=rf"$Q_x^' = {twiss0.dqx:.2f}, Q_y^' = {twiss0.dqy:.2f}$")
ax2.set_ylim(bottom=0)
ax2.set_ylabel("Intensity [$10^{11}$ppb]")
ax2.set_xlabel("Turn")

plt.gcf().align_ylabels((ax1, ax2))
ax1.legend(ncols=2, loc="lower center", bbox_to_anchor=(0.5, 1))
plt.subplots_adjust(left=0.2, hspace=0.3, top=0.9)
plt.show()

### Adjust the Sextupoles to Change the Chromaticity

In [ ]:
%%capture
line.enable_time_dependent_vars = False
opt = line.match(
    solve=False,
    method="4d",
    vary=xt.VaryList(["ksf", "ksd"], step=1e-3),
    targets=xt.TargetSet(dqx=-4.0, dqy=-4.0, tol=1e-3, tag="chrom"),
)
opt.solve()
twiss1 = line.twiss()

### Repeat the Simulation

In [ ]:
bunch = bunch0.copy(_context=context)

In [ ]:
bunch.x += 1e-3
line.enable_time_dependent_vars = True
line.track(bunch, log=track_log, num_turns=1000, with_progress=5)

In [ ]:
# Plot logged data
x_mean_corr = np.array(line.log_last_track["x_mean"])
intensity_corr = np.array(line.log_last_track["intensity"])

plt.figure(figsize=(7, 5))
ax1 = plt.subplot(2, 1, 1)
ax1.plot(x_mean, label=rf"$Q_x^' = {twiss0.dqx:.2f}, Q_y^' = {twiss0.dqy:.2f}$")
ax1.plot(x_mean_corr, label=rf"$Q_x^' = {twiss1.dqx:.2f}, Q_y^' = {twiss1.dqy:.2f}$")
ax1.set_ylabel(r"$x_{\mathrm{centroid}}$ [m]")

ax2 = plt.subplot(2, 1, 2, sharex=ax1)
ax2.plot(intensity * 1e-11, label=rf"$Q_x^' = {twiss0.dqx:.2f}, Q_y^' = {twiss0.dqy:.2f}$")
ax2.plot(intensity_corr * 1e-11, label=rf"$Q_x^' = {twiss1.dqx:.2f}, Q_y^' = {twiss1.dqy:.2f}$")
ax2.set_ylim(bottom=0)
ax2.set_ylabel("Intensity [$10^{11}$ppb]")
ax2.set_xlabel("Turn")

plt.subplots_adjust(left=0.2, hspace=0.3, top=0.9)
ax1.legend(ncols=2, loc="lower center", bbox_to_anchor=(0.5, 1))
plt.show()

---